In [2]:
#Install Pyrsgis
!pip install pyrsgis

In [1]:
#Get dataset
!git clone https://github.com/ridhodwidharmawan/Builtup-area-Landsat-ANN

Cloning into 'Builtup-area-Landsat-ANN'...


In [2]:
import os
import numpy as np
from tensorflow import keras
from pyrsgis import raster
from pyrsgis.convert import changeDimension
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

Warning! matplotlib_scalebar library not found.


In [3]:
# Change the directory
os.chdir("Builtup-area-Landsat-ANN\dataset")

In [4]:
# Assign file names
subset_yogya = 'subset_l8_yogya.tif'
builtup_yogya = 'builtup_yogya.tif'
subset_jateng = 'subset_l8_jateng.tif'

In [5]:
# Read the rasters as array
ds1, featuresyogya = raster.read(subset_yogya, bands='all')
ds2, labelyogya = raster.read(builtup_yogya, bands=1)
ds3, featuresjateng = raster.read(subset_jateng, bands='all')

In [6]:
# Print the size of the arrays
print("Yogya multispectral image shape: ", featuresyogya.shape)
print("Yogya  builtup label image shape: ", labelyogya.shape)
print("Jateng multispectral image shape: ", featuresjateng.shape)

Yogya multispectral image shape:  (6, 844, 843)
Yogya  builtup label image shape:  (844, 843)
Jateng multispectral image shape:  (6, 4052, 3292)


In [7]:
# Clean the labelled data to replace NoData values by zero
labelyogya = (labelyogya == 1).astype(int)

In [9]:
# Reshape the array to single dimensional array
featuresyogya = changeDimension(featuresyogya)
labelyogya = changeDimension (labelyogya)
featuresjateng = changeDimension(featuresjateng)
nBands = featuresyogya.shape[1]

In [10]:
# Print new dimension size
print("Yogya multispectral image shape: ", featuresyogya.shape)
print("Yogya builtup label image shape: ", labelyogya.shape)
print("Jateng multispectral image shape: ", featuresjateng.shape)

Yogya multispectral image shape:  (711492, 6)
Yogya builtup label image shape:  (711492,)
Jateng multispectral image shape:  (13339184, 6)


In [12]:
# Split testing and training datasets
xTrain, xTest, yTrain, yTest = train_test_split(featuresyogya, labelyogya, test_size=0.4, random_state=42)

In [13]:
# Print train size
print(xTrain.shape)
print(yTrain.shape)

(426895, 6)
(426895,)


In [14]:
# Print test size
print(xTest.shape)
print(yTest.shape)

(284597, 6)
(284597,)


In [15]:
# Normalise the data
xTrain = xTrain / 10000
xTest = xTest / 10000
featuresjateng = featuresjateng / 10000

In [16]:
# Reshape the data
xTrain = xTrain.reshape((xTrain.shape[0], 1, xTrain.shape[1]))
xTest = xTest.reshape((xTest.shape[0], 1, xTest.shape[1]))
featuresjateng = featuresjateng.reshape((featuresjateng.shape[0], 1, featuresjateng.shape[1]))

In [17]:
# Print the shape of reshaped data
print(xTrain.shape, xTest.shape, featuresjateng.shape)

(426895, 1, 6) (284597, 1, 6) (13339184, 1, 6)


In [18]:
# Define the parameters of the model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1, nBands)),
    keras.layers.Dense(14, activation='relu'),
    keras.layers.Dense(2, activation='softmax')])

In [19]:
# Define the accuracy metrics and parameters
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [20]:
# Run the model
model.fit(xTrain, yTrain, epochs=2)

Epoch 1/2
13341/13341 [==============================] - 24s 2ms/step - loss: 0.4687 - accuracy: 0.7855
Epoch 2/2
13341/13341 [==============================] - 24s 2ms/step - loss: 0.4247 - accuracy: 0.8177


In [21]:
# Predict for test data 
yTestPredicted = model.predict(xTest)
yTestPredicted = yTestPredicted[:,1]

In [22]:
# Calculate and display the error metrics
yTestPredicted = (yTestPredicted>0.5).astype(int)
cMatrix = confusion_matrix(yTest, yTestPredicted)
pScore = precision_score(yTest, yTestPredicted)
rScore = recall_score(yTest, yTestPredicted)

In [23]:
print("Confusion matrix: for 14 nodes\n", cMatrix)
print("\nP-Score: %.3f, R-Score: %.3f" % (pScore, rScore))

Confusion matrix: for 14 nodes
 [[141472  23111]
 [ 27861  92153]]

P-Score: 0.799, R-Score: 0.768


In [25]:
predicted = model.predict(featuresjateng)
predicted = predicted[:,1]

In [26]:
#Export raster
prediction = np.reshape(predicted, (ds3.RasterYSize, ds3.RasterXSize))
outFile = 'builtup_predicted_l8_jateng_2018.tif'
raster.export(prediction, ds3, filename=outFile, dtype='float')